In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('fingerprints_data.csv')

In [3]:
#removing NaN values, if there is any
df = df.dropna()

In [4]:
df.shape

(1650, 3241)

In [5]:
fingerprint_array = df[df.drop(['Standard Type', 'Standard Value'], axis=1).columns].values

In [6]:
df_ic50 = df[df['Standard Type'] == 'IC50']
df_ki = df[df['Standard Type'] == 'Ki']

In [7]:
#removing feature with low variance
from sklearn.feature_selection import VarianceThreshold

In [8]:
selector = VarianceThreshold(threshold=0.01)
X_ic50 = df_ic50.drop(columns=['Standard Type', 'Standard Value', 'Smiles'])
X_ki = df_ki.drop(columns=['Standard Type', 'Standard Value', 'Smiles'])

# Define original_feature_names from the dataframe columns
original_feature_names = X_ic50.columns.tolist()

X_ic50 = selector.fit_transform(X_ic50)
selector_indices_ic50 = selector.get_support(indices=True)
selector_feature_names0 = np.array([original_feature_names[i] for i in selector_indices_ic50])

X_ki = selector.fit_transform(X_ki)

selector_indices_ki50 = selector.get_support(indices=True)
selector_feature_names1 = np.array([original_feature_names[i] for i in selector_indices_ki50])

In [9]:
print(len(X_ic50[0]), len(X_ki[0]))

1792 1710


In [10]:
#univariate feature selection, with mutual information regression
from sklearn.feature_selection import SelectKBest, mutual_info_regression

In [11]:
selector = SelectKBest(score_func=mutual_info_regression, k=2000)
y_ic50 = df_ic50['Standard Value']
y_ki = df_ki['Standard Value']
X_ic50 = selector.fit_transform(X_ic50, y_ic50)

selector_indices0_ic50 = selector.get_support(indices=True)
selector_feature_names2 = [selector_feature_names0[i] for i in selector_indices0_ic50]

X_ki = selector.fit_transform(X_ki, y_ki)

selector_indices0_ki = selector.get_support(indices=True)
selector_feature_names3 = [selector_feature_names1[i] for i in selector_indices0_ki]

c:\Users\SUJAL GULIA\anaconda3\envs\amyloid_venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:783: UserWarning: k=2000 is greater than n_features=1792. All the features will be returned.
  warnings.warn(
c:\Users\SUJAL GULIA\anaconda3\envs\amyloid_venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:783: UserWarning: k=2000 is greater than n_features=1710. All the features will be returned.
  warnings.warn(


In [12]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [13]:
pipeline_ic50 = make_pipeline(
    StandardScaler(),  #feature scaling is required for Lasso
    LassoCV(cv=5, random_state=42) #applying LASSO with 5 fold cross validation
)

pipeline_ic50.fit(X_ic50, y_ic50)

lasso_mask_ic50 = pipeline_ic50.named_steps['lassocv'].coef_ != 0
lasso_feature_names_ic50 = [selector_feature_names2[i] for i, keep in enumerate(lasso_mask_ic50) if keep]
X_ic50_selected = X_ic50[:, lasso_mask_ic50]

c:\Users\SUJAL GULIA\anaconda3\envs\amyloid_venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.429e+08, tolerance: 2.390e+08
  model = cd_fast.enet_coordinate_descent(
c:\Users\SUJAL GULIA\anaconda3\envs\amyloid_venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.647e+08, tolerance: 2.601e+08
  model = cd_fast.enet_coordinate_descent(


In [14]:
pipeline_ki = make_pipeline(
    StandardScaler(),
    LassoCV(cv=5, random_state=42)
)
pipeline_ki.fit(X_ki, y_ki)

lasso_mask_ki = pipeline_ki.named_steps['lassocv'].coef_ != 0
lasso_feature_names_ki = [selector_feature_names3[i] for i, keep in enumerate(lasso_mask_ki) if keep]
X_ki_selected = X_ki[:, lasso_mask_ki]

In [15]:
df_ic50 = pd.DataFrame(X_ic50_selected, columns=lasso_feature_names_ic50)
df_ic50['Standard Value'] = y_ic50.values
df_ic50.to_csv('final_ic50.csv', index=False)

df_ki = pd.DataFrame(X_ki_selected, columns=lasso_feature_names_ki)
df_ki['Standard Value'] = y_ki.values
df_ki.to_csv('final_ki.csv', index=False)

In [17]:
print(len(X_ic50_selected[0]), len(X_ki_selected[0]))

25 37
